# TodoApp — Notebook CLI

Use this notebook to create the database, seed the sample todos, add new todos, and list todos — all from cells instead of the command line.

Run each code cell in order.

In [7]:
import sqlite3
import os

# Database file path
DB_PATH = 'todos.db'

# Connect to the database
def get_connection():
    """Get a connection to the SQLite database."""
    return sqlite3.connect(DB_PATH)

conn = get_connection()
print(f'Connected to database: {DB_PATH}')

Connected to database: todos.db


In [8]:
# Create the todos table using SQL
conn = get_connection()
cursor = conn.cursor()

# SQL query to create the table
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS todos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    priority INTEGER DEFAULT 1,
    complete BOOLEAN DEFAULT 0
)
"""

cursor.execute(CREATE_TABLE_SQL)
conn.commit()
print('Table "todos" created (if it did not exist)')
print('\nSQL Query used:')
print(CREATE_TABLE_SQL)

Table "todos" created (if it did not exist)

SQL Query used:

CREATE TABLE IF NOT EXISTS todos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    priority INTEGER DEFAULT 1,
    complete BOOLEAN DEFAULT 0
)



In [9]:
# Sample data matching the table in the repo
SAMPLE_TODOS = [
    ("Go to store", "To pick up eggs", 4, False),
    ("Haircut", "Need to get length 1mm", 3, False),
    ("Feed dog", "Make sure to use new food brand", 5, False),
    ("Water plant", "Inside and Outside plants", 4, False),
    ("Learn something new", "Learn to program", 5, False),
]

def seed_db(force=False):
    """Insert sample todos into the database."""
    conn = get_connection()
    cursor = conn.cursor()
    
    # Check if table has existing data
    cursor.execute("SELECT COUNT(*) FROM todos")
    existing = cursor.fetchone()[0]
    
    if existing and not force:
        print(f'Skipping seed: table already has {existing} rows. Use force=True to insert anyway.')
        return
    
    # Delete existing rows if force=True
    if force and existing:
        cursor.execute("DELETE FROM todos")
        print(f'Deleted {existing} existing rows')
    
    # SQL INSERT query
    INSERT_SQL = "INSERT INTO todos (title, description, priority, complete) VALUES (?, ?, ?, ?)"
    
    for todo in SAMPLE_TODOS:
        cursor.execute(INSERT_SQL, todo)
    
    conn.commit()
    conn.close()
    
    print(f'Seeded {len(SAMPLE_TODOS)} todos')
    print('\nSQL Query used:')
    print(INSERT_SQL)

In [10]:
def add_todo(title, description='', priority=1, complete=False):
    """Add a single todo to the database."""
    conn = get_connection()
    cursor = conn.cursor()
    
    # SQL INSERT query
    INSERT_SQL = "INSERT INTO todos (title, description, priority, complete) VALUES (?, ?, ?, ?)"
    
    cursor.execute(INSERT_SQL, (title, description, priority, int(complete)))
    conn.commit()
    
    # Get the inserted ID
    new_id = cursor.lastrowid
    conn.close()
    
    print(f'Added todo id={new_id} title={title}')
    print('\nSQL Query used:')
    print(INSERT_SQL)

# Example usage:
# add_todo('Buy milk', '2 liters', priority=2)

In [17]:
def list_todos():
    """List all todos from the database."""
    conn = get_connection()
    cursor = conn.cursor()
    
    # SQL SELECT query
    SELECT_SQL = "SELECT id, title, description, priority, complete FROM todos ORDER BY id"
    
    cursor.execute(SELECT_SQL)
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print('No todos found. Use seed_db() or add_todo() to create entries.')
        return []
    
    # print(f"{'Id':<3} {'Title':<22} {'Description':<32} {'Priority':<8} {'Complete':<8}")
    # print('-' * 90)
    # for r in rows:
    #     row_id, title, description, priority, complete = r
    #     print(f"{row_id:<3} {title:<22} {description:<32} {priority!s:<8} {int(complete):<8}")
    
    # print('\nSQL Query used:')
    # print(SELECT_SQL)
    
    return rows

# Example: run list_todos() to see entries
# list_todos()  # uncomment and run to list

Notes:
- Use the `seed_db()` cell to populate initial rows.
- Use `add_todo(...)` to add single entries.
- Use `list_todos()` to view the table.
- This avoids running commands in the OS shell; just run notebook cells interactively.

In [20]:
def query_with_headers(sql_query, params=()):
    """Execute a query and display results with column headers using cursor.description."""
    conn = get_connection()
    cursor = conn.cursor()
    
    cursor.execute(sql_query, params)
    
    # Get column names from cursor.description
    columns = [description[0] for description in cursor.description]
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print('No results found.')
        return []
    
    # Print headers
    header = ' | '.join(f'{col:<20}' for col in columns)
    print(header)
    print('-' * len(header))
    
    # Print rows
    for row in rows:
        print(' | '.join(f'{str(val):<20}' for val in row))
    
    return rows

# Example usage - query directly with headers:
# query_with_headers("SELECT * FROM todos ORDER BY id")

In [22]:
query_with_headers("SELECT * FROM todos ORDER BY id")

id                   | title                | description          | priority             | complete            
----------------------------------------------------------------------------------------------------------------
1                    | Go to store          | To pick up eggs      | 4                    | 0                   
2                    | Haircut              | Need to get length 1mm | 3                    | 0                   
3                    | Feed dog             | Make sure to use new food brand | 5                    | 0                   
4                    | Water plant          | Inside and Outside plants | 4                    | 0                   
5                    | Learn something new  | Learn to program     | 5                    | 0                   


[(1, 'Go to store', 'To pick up eggs', 4, 0),
 (2, 'Haircut', 'Need to get length 1mm', 3, 0),
 (3, 'Feed dog', 'Make sure to use new food brand', 5, 0),
 (4, 'Water plant', 'Inside and Outside plants', 4, 0),
 (5, 'Learn something new', 'Learn to program', 5, 0)]